In [ ]:
import pandas as pd
import numpy as np
from brainflow.board_shim import BoardShim, BrainFlowInputParams, LogLevels, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes
import matplotlib
import os
import torch
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from moviepy.video.io.bindings import mplfig_to_npimage
import cv2
from torch.utils.data import DataLoader
from PIL import Image
import time

# matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy import signal
%matplotlib inline

In [ ]:
board_id = 38
eeg_names = ['TP9', 'Fp1', 'Fp2', 'TP10', '_AUX']
sf = 256

In [ ]:
scaler = StandardScaler()
dataset_dir = 'D:\\Nicko\\TUGAS_AKHIR\\Dataset_new\\Dataset_TA\\'
destination_dir = 'D:\\Nicko\\TUGAS_AKHIR\\Dataset_new\\Dataset_TA_img\\'
file_list_dir = dataset_dir + 'csv_list.csv'

In [ ]:
file_lists = pd.read_csv(file_list_dir, header=None)

In [ ]:
for idx,csv_name in enumerate(file_lists[0]):
  fig, axes = plt.subplots(1,5, figsize=(10.8,7.2))
  plt.subplots_adjust(wspace=0, hspace=0)
  for i in range(5):
    axes[i].get_xaxis().set_visible(False)
    axes[i].get_yaxis().set_visible(False)
  start = time.time()
  csv_name = csv_name.replace('/','\\')
  print(idx, csv_name)
  file_path = os.path.join(dataset_dir, csv_name)
  img_destination = os.path.join(destination_dir, csv_name[:-4])
  os.mkdir(img_destination)

  #load csv
  data = DataFilter.read_file(file_path)
  eeg_channels = BoardShim.get_eeg_channels(board_id)
  eeg_channels.append(5)
  eeg_data = data[eeg_channels, :]
  eeg_data = eeg_data/1000000

  #bandpass filter 8-30hz
  for i in range(5):
    DataFilter.perform_bandpass(eeg_data[i], sf, 8, 30, 2,
                              FilterTypes.BUTTERWORTH.value, 0)

  #normalized
  scaled = scaler.fit_transform(eeg_data)

  #divide to 2 seconds
  list_eeg_data = []
  for i in range(5):
    temp = []
    for j in range(5):
      temp.append(scaled[j][i*(sf*2):(i+1)*(sf*2)])
    list_eeg_data.append(temp)

  #build spectrogram (1 gambar = 1 channel)
  win = 1 * sf

  spectrograms = []
  for i in range(5):
    for j in range(5):
      #j = tiap 2 detik, i = channelnya
      f,t,Zxx = signal.stft(list_eeg_data[j][i], fs=sf, window='hann', nperseg=win)
      axes[j].pcolormesh(t, f, np.abs(Zxx), shading='gouraud', cmap='gray')

    numpy_fig = mplfig_to_npimage(fig)

    x = 136
    y = 98
    w = 836
    h = 532
    reduced_dim = cv2.cvtColor(numpy_fig[y:y+h,x:x+w], cv2.COLOR_BGR2GRAY)
    cropped_img = reduced_dim[352:,:]
    filename = eeg_names[i] + '.png'
    cv2.imwrite(img_destination+'\\'+filename, cropped_img)
  plt.close()
  end = time.time()
  print('Elapsed Time : ', end - start)